# Dataset







__Overall__  
We are going to use foursquare location data to find top 100 american restaurants in San Francisco

__For problem 1__  
We are going to take the first 30 records to represent our current list of customers

__For problem 2__  
We are going to take the first 90 records to represent our future list of customers 

## This is the code

In [2]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

from matplotlib import pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs

print('Folium installed')
print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.8.4

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.8.4

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Folium installed
Libraries imported.


In [6]:
CLIENT_ID = 'FJWQK20PTI3RNPU3CLXODPPMNX4WPVBS0UTJBW5X20HKD2P0' # your Foursquare ID
CLIENT_SECRET = '3NPUXRCEJ31MOZS1D3ZF0CWKEWHPXH4BCOKMBHE2ODTN13TI' # your Foursquare Secret
VERSION = '20180610'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + 'CLIENT_ID')
print('CLIENT_SECRET:' + 'CLIENT_SECRET')

Your credentails:
CLIENT_ID: CLIENT_ID
CLIENT_SECRET:CLIENT_SECRET


In [7]:
address = 'San francisco, USA'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

37.7790262 -122.4199061


In [12]:
search_query = 'restaurant'
category_Id = '4bf58dd8d48988d14e941735'
radius = 5000
LIMIT = 90
VERSION='20180610'

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    latitude, 
    longitude, 
    VERSION, 
    radius, 
    LIMIT,
    category_Id)
url

'https://api.foursquare.com/v2/venues/explore?client_id=FJWQK20PTI3RNPU3CLXODPPMNX4WPVBS0UTJBW5X20HKD2P0&client_secret=3NPUXRCEJ31MOZS1D3ZF0CWKEWHPXH4BCOKMBHE2ODTN13TI&ll=37.7790262,-122.4199061&v=20180610&radius=5000&limit=90&categoryId=4bf58dd8d48988d14e941735'

In [13]:
results = requests.get(url).json()

In [10]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [28]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
#filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
filtered_columns = ['venue.name', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
#nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
#nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.columns = ['Customer', 
                  'Latitude', 
                  'Longitude']

nearby_venues.head()

,Customer,Latitude,Longitude
0,The Progress,37.783745,-122.432972
1,Octavia,37.787935,-122.426934
2,Dottie's True Blue Cafe,37.781748,-122.409829
3,Box Kitchen,37.781158,-122.406243
4,Morty's Delicatessen,37.781710,-122.415243


In [16]:
nearby_venues.shape

(90, 3)

In [27]:
current_cust=nearby_venues.head(30)

current_cust.shape

(30, 3)

In [29]:
current_cust.head()

,Customer,Latitude,Longitude
0,The Progress,37.783745,-122.432972
1,Octavia,37.787935,-122.426934
2,Dottie's True Blue Cafe,37.781748,-122.409829
3,Box Kitchen,37.781158,-122.406243
4,Morty's Delicatessen,37.781710,-122.415243


In [33]:
map_current = folium.Map(location=[latitude, longitude], zoom_start=13)

for lat, lng, label in zip(current_cust['Latitude'], current_cust['Longitude'], current_cust['Customer']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_current)  

map_current


In [24]:
future_cust=nearby_venues

future_cust.shape

(90, 3)

In [25]:
future_cust

,Customer,Latitude,Longitude
0,The Progress,37.783745,-122.432972
1,Octavia,37.787935,-122.426934
2,Dottie's True Blue Cafe,37.781748,-122.409829
3,Box Kitchen,37.781158,-122.406243
4,Morty's Delicatessen,37.781710,-122.415243
...,...,...,...
85,Eats,37.783210,-122.460090
86,Prospect,37.789823,-122.391007
87,Fog City,37.803539,-122.401881
88,Delancey Street Restaurant,37.784568,-122.388457


In [31]:
map_future = folium.Map(location=[latitude, longitude], zoom_start=13)

for lat, lng, label in zip(future_cust['Latitude'], future_cust['Longitude'], future_cust['Customer']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_future)  

map_future